In [85]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import RandomOverSampler, SMOTE
import dataframe_image as dfi

https://www.analyticsvidhya.com/blog/2017/03/imbalanced-data-classification/
https://stackoverflow.com/questions/55921286/should-i-balance-the-test-set-when-i-have-highly-unbalanced-data


In [86]:
df = pd.read_csv("../data/final/trans_fraud_extra_2022-10-31_12:38:18.csv")
df['normAmount'] = StandardScaler().fit_transform(df['Amount'].values.reshape (-1,1))
df = df.drop(['Amount'], axis = 1)

df.head()


,Unnamed: 0,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V25,V26,V27,V28,Class,inversed_dist,fraud_neighbor_count,community_risk,personalized_page_rank,normAmount
0,0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.128539,-0.189115,0.133558,-0.021053,0,0.0,0,0.002786,0.000177,0.244964
1,1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,0.167170,0.125895,-0.008983,0.014724,0,0.0,0,0.000000,0.000359,-0.342475
2,2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,-0.327642,-0.139097,-0.055353,-0.059752,0,0.0,0,0.000000,0.000047,1.160686
3,3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,0.647376,-0.221929,0.062723,0.061458,0,0.0,0,0.001603,0.000064,0.140534
4,4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,-0.206010,0.502292,0.219422,0.215153,0,0.0,0,0.002786,0.000136,-0.073403


In [87]:
result_dir = "../results"
results_name = "all_previous_in_training"

In [88]:
GROUP_TIME_LENGTH = 7200
max_time = df["Time"].max()
group_df = df.groupby(pd.cut(df["Time"], np.arange(-1, max_time + GROUP_TIME_LENGTH, GROUP_TIME_LENGTH)))


Distribution of Target class

In [89]:
df["Class"].value_counts()

0    284315
1       492
Name: Class, dtype: int64

Defining classifier

In [90]:
def predict_with_metric(model,X_test, y_test):
    prediction = model.predict(X_test)
    conf_matrix = confusion_matrix(y_test,prediction)
    tn= conf_matrix[0][0]
    fp= conf_matrix[0][1]
    fn = conf_matrix[1][0]
    tp =conf_matrix[1][1]
    return {"Prediction":list(prediction.flatten()),"Accuracy":accuracy_score(y_test, prediction),"Precision":precision_score(y_test, prediction),"Recall":recall_score(y_test,prediction),"F1":f1_score(y_test,prediction),"TN":tn,"FP":fp,"FN":fn,"TP":tp}

In [91]:
%%time
NUM_ESTIMATOR = 50
rf_clf = RandomForestClassifier(n_estimators=NUM_ESTIMATOR)

CPU times: user 30 µs, sys: 1 µs, total: 31 µs
Wall time: 31.9 µs


In [92]:
normal_acc_l = []
normal_pre_l = []
normal_rec_l = []
normal_f1_l = []
normal_tn_l= []
normal_fp_l= []
normal_fn_l = []
normal_tp_l = []
group_time_l = []
train_data = pd.DataFrame()
for group_time, group in group_df:
    # print("")
    print(f"===== Processing group:{group_time} , group number:{int((group_time.left + 1) / GROUP_TIME_LENGTH + 1)}=====")
    group_number = int((group_time.left + 1) / GROUP_TIME_LENGTH + 1)
    if group_number > 3:
        print(f"Predicting group {group_number}")
        X_test = group[group.columns.difference(["Class","Time"])]
        y_train = group["Class"]
        model_prediction  = predict_with_metric(rf_clf,X_test,y_train)
        group_time_l.append(group_time)
        normal_acc_l.append(model_prediction["Accuracy"])
        normal_pre_l.append(model_prediction["Precision"])
        normal_rec_l.append(model_prediction["Recall"])
        normal_f1_l.append(model_prediction["F1"])
        normal_tn_l.append(model_prediction["TN"])
        normal_fp_l.append(model_prediction["FP"])
        normal_fn_l.append(model_prediction["FN"])
        normal_tp_l.append(model_prediction["TP"])
    train_data = pd.concat([train_data,group])
    X_train = train_data[train_data.columns.difference(["Class","Time"])]
    y_train = train_data["Class"]
    rf_clf.fit(X_train,y_train)

===== Processing group:(-1.0, 7199.0] , group number:1=====
===== Processing group:(7199.0, 14399.0] , group number:2=====
===== Processing group:(14399.0, 21599.0] , group number:3=====
===== Processing group:(21599.0, 28799.0] , group number:4=====
Predicting group 4
===== Processing group:(28799.0, 35999.0] , group number:5=====
Predicting group 5
===== Processing group:(35999.0, 43199.0] , group number:6=====
Predicting group 6
===== Processing group:(43199.0, 50399.0] , group number:7=====
Predicting group 7
===== Processing group:(50399.0, 57599.0] , group number:8=====
Predicting group 8
===== Processing group:(57599.0, 64799.0] , group number:9=====
Predicting group 9
===== Processing group:(64799.0, 71999.0] , group number:10=====
Predicting group 10
===== Processing group:(71999.0, 79199.0] , group number:11=====
Predicting group 11
===== Processing group:(79199.0, 86399.0] , group number:12=====
Predicting group 12
===== Processing group:(86399.0, 93599.0] , group number:13=

In [93]:
metric_data = list(zip(group_time_l,normal_acc_l,normal_pre_l,normal_rec_l,normal_f1_l,normal_tn_l,normal_fp_l,normal_fn_l,normal_tp_l))
df = pd.DataFrame(metric_data, columns =['period','Accuracy', 'Precision', 'Recall','F-1','TN','FP','FN','TP'])
df.to_csv(f"{result_dir}/dynamic_clf_normal_extra_feature_{results_name}.csv")
dfi.export(df.style.hide_index(), f"{result_dir}/dynamic_clf_normal_extra_feature_{results_name}.png")
df

/tmp/ipykernel_3720163/117895703.py:4: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  dfi.export(df.style.hide_index(), f"{result_dir}/dynamic_clf_normal_extra_feature_{results_name}.png")
[1113/134055.952630:WARNING:sandbox_linux.cc(380)] InitializeSandbox() called with multiple threads in process gpu-process.
[1113/134055.958656:WARNING:bluez_dbus_manager.cc(247)] Floss manager not present, cannot set Floss enable/disable.
[1113/134056.114582:INFO:headless_shell.cc(660)] Written to file /tmp/tmpfu_wk3e7/temp.png.


,period,Accuracy,Precision,Recall,F-1,TN,FP,FN,TP
0,"(21599.0, 28799.0]",0.997500,0.709677,0.846154,0.771930,5164,9,4,22
1,"(28799.0, 35999.0]",0.999311,0.923077,0.600000,0.727273,13036,1,8,12
2,"(35999.0, 43199.0]",0.999524,0.930233,0.888889,0.909091,16757,3,5,40
3,"(43199.0, 50399.0]",0.999282,0.733333,0.611111,0.666667,15295,4,7,11
4,"(50399.0, 57599.0]",0.999685,1.000000,0.814815,0.897959,15838,0,5,22
5,"(57599.0, 64799.0]",0.999617,1.000000,0.769231,0.869565,15642,0,6,20
6,"(64799.0, 71999.0]",0.999578,1.000000,0.681818,0.810811,16579,0,7,15
7,"(71999.0, 79199.0]",0.999417,0.739130,0.772727,0.755556,18847,6,5,17
8,"(79199.0, 86399.0]",0.999734,0.944444,0.850000,0.894737,15038,1,3,17
9,"(86399.0, 93599.0]",0.997908,0.500000,0.500000,0.500000,5717,6,6,6


## Random Over sampling

In [94]:
%%time
NUM_ESTIMATOR = 50
rf_clf = RandomForestClassifier(n_estimators=NUM_ESTIMATOR)

CPU times: user 113 µs, sys: 0 ns, total: 113 µs
Wall time: 119 µs


In [95]:
ros_acc_l = []
ros_pre_l = []
ros_rec_l = []
ros_f1_l = []
ros_tn_l= []
ros_fp_l= []
ros_fn_l = []
ros_tp_l = []
group_time_l = []
train_data = pd.DataFrame()
for group_time, group in group_df:
    # print("")
    print(f"===== Processing group:{group_time} , group number:{int((group_time.left + 1) / GROUP_TIME_LENGTH + 1)}=====")
    group_number = int((group_time.left + 1) / GROUP_TIME_LENGTH + 1)
    if group_number > 3:
        print(f"Predicting group {group_number}")
        X_test = group[group.columns.difference(["Class","Time"])]
        y_train = group["Class"]
        model_prediction  = predict_with_metric(rf_clf,X_test,y_train)
        group_time_l.append(group_time)
        ros_acc_l.append(model_prediction["Accuracy"])
        ros_pre_l.append(model_prediction["Precision"])
        ros_rec_l.append(model_prediction["Recall"])
        ros_f1_l.append(model_prediction["F1"])
        ros_tn_l.append(model_prediction["TN"])
        ros_fp_l.append(model_prediction["FP"])
        ros_fn_l.append(model_prediction["FN"])
        ros_tp_l.append(model_prediction["TP"])
    # Random over sampling
    train_data = pd.concat([train_data,group])
    X_train = train_data[train_data.columns.difference(["Class","Time"])]
    y_train = train_data["Class"]
    ros = RandomOverSampler(random_state=1234)
    X_group_ros, y_group_ros = ros.fit_resample(X_train,y_train)
    # Check the number of records after over sampling
    print(sorted(Counter(y_group_ros).items()))
    # Training model
    rf_clf.fit(X_train,y_train)


===== Processing group:(-1.0, 7199.0] , group number:1=====
[(0, 6176), (1, 6176)]
===== Processing group:(7199.0, 14399.0] , group number:2=====
[(0, 9539), (1, 9539)]
===== Processing group:(14399.0, 21599.0] , group number:3=====
[(0, 12285), (1, 12285)]
===== Processing group:(21599.0, 28799.0] , group number:4=====
Predicting group 4
[(0, 17458), (1, 17458)]
===== Processing group:(28799.0, 35999.0] , group number:5=====
Predicting group 5
[(0, 30495), (1, 30495)]
===== Processing group:(35999.0, 43199.0] , group number:6=====
Predicting group 6
[(0, 47255), (1, 47255)]
===== Processing group:(43199.0, 50399.0] , group number:7=====
Predicting group 7
[(0, 62554), (1, 62554)]
===== Processing group:(50399.0, 57599.0] , group number:8=====
Predicting group 8
[(0, 78392), (1, 78392)]
===== Processing group:(57599.0, 64799.0] , group number:9=====
Predicting group 9
[(0, 94034), (1, 94034)]
===== Processing group:(64799.0, 71999.0] , group number:10=====
Predicting group 10
[(0, 1106

In [96]:
metric_data = list(zip(group_time_l,ros_acc_l,ros_pre_l,ros_rec_l,ros_f1_l,ros_tn_l,ros_fp_l,ros_fn_l,ros_tp_l))
df = pd.DataFrame(metric_data, columns =['period','Accuracy', 'Precision', 'Recall','F-1','TN','FP','FN','TP'])
df.to_csv(f"{result_dir}/dynamic_clf_ros_extra_feature_{results_name}.csv")
dfi.export(df.style.hide_index(), f"{result_dir}/dynamic_clf_ros_extra_feature_{results_name}.png")
df

/tmp/ipykernel_3720163/3529978618.py:4: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  dfi.export(df.style.hide_index(), f"{result_dir}/dynamic_clf_ros_extra_feature_{results_name}.png")
[1113/135502.887907:WARNING:sandbox_linux.cc(380)] InitializeSandbox() called with multiple threads in process gpu-process.
[1113/135502.899603:WARNING:bluez_dbus_manager.cc(247)] Floss manager not present, cannot set Floss enable/disable.
[1113/135503.013752:INFO:headless_shell.cc(660)] Written to file /tmp/tmppxs73_c7/temp.png.


,period,Accuracy,Precision,Recall,F-1,TN,FP,FN,TP
0,"(21599.0, 28799.0]",0.996922,0.647059,0.846154,0.733333,5161,12,4,22
1,"(28799.0, 35999.0]",0.999464,0.933333,0.700000,0.800000,13036,1,6,14
2,"(35999.0, 43199.0]",0.999643,0.933333,0.933333,0.933333,16757,3,3,42
3,"(43199.0, 50399.0]",0.999282,0.769231,0.555556,0.645161,15296,3,8,10
4,"(50399.0, 57599.0]",0.999685,1.000000,0.814815,0.897959,15838,0,5,22
5,"(57599.0, 64799.0]",0.999617,1.000000,0.769231,0.869565,15642,0,6,20
6,"(64799.0, 71999.0]",0.999578,1.000000,0.681818,0.810811,16579,0,7,15
7,"(71999.0, 79199.0]",0.999364,0.750000,0.681818,0.714286,18848,5,7,15
8,"(79199.0, 86399.0]",0.999734,0.944444,0.850000,0.894737,15038,1,3,17
9,"(86399.0, 93599.0]",0.998082,0.538462,0.583333,0.560000,5717,6,5,7


# Over sampling with SMOTHE

In [97]:
%%time
NUM_ESTIMATOR = 50
rf_clf = RandomForestClassifier(n_estimators=NUM_ESTIMATOR)

CPU times: user 88 µs, sys: 1e+03 ns, total: 89 µs
Wall time: 91.1 µs


In [98]:
smote_acc_l = []
smote_pre_l = []
smote_rec_l = []
smote_f1_l = []
smote_tn_l= []
smote_fp_l= []
smote_fn_l = []
smote_tp_l = []
group_time_l = []
train_data = pd.DataFrame()
for group_time, group in group_df:
    # print("")
    print(f"===== Processing group:{group_time} , group number:{int((group_time.left + 1) / GROUP_TIME_LENGTH + 1)}=====")
    group_number = int((group_time.left + 1) / GROUP_TIME_LENGTH + 1)
    if group_number > 3:
        print(f"Predicting group {group_number}")
        X_test = group[group.columns.difference(["Class","Time"])]
        y_train = group["Class"]
        model_prediction  = predict_with_metric(rf_clf,X_test,y_train)
        group_time_l.append(group_time)
        smote_acc_l.append(model_prediction["Accuracy"])
        smote_pre_l.append(model_prediction["Precision"])
        smote_rec_l.append(model_prediction["Recall"])
        smote_f1_l.append(model_prediction["F1"])
        smote_tn_l.append(model_prediction["TN"])
        smote_fp_l.append(model_prediction["FP"])
        smote_fn_l.append(model_prediction["FN"])
        smote_tp_l.append(model_prediction["TP"])
    # Random over sampling
    train_data = pd.concat([train_data,group])
    X_train = train_data[train_data.columns.difference(["Class","Time"])]
    y_train = train_data["Class"]
    smote = SMOTE(random_state=1234,k_neighbors=3)
    X_group_smote, y_group_smote = smote.fit_resample(X_train,y_train)
    # Check the number of records after over sampling
    print(sorted(Counter(y_group_smote).items()))
    # Training model
    rf_clf.fit(X_train,y_train)
    ## pdate train data


===== Processing group:(-1.0, 7199.0] , group number:1=====
[(0, 6176), (1, 6176)]
===== Processing group:(7199.0, 14399.0] , group number:2=====
[(0, 9539), (1, 9539)]
===== Processing group:(14399.0, 21599.0] , group number:3=====
[(0, 12285), (1, 12285)]
===== Processing group:(21599.0, 28799.0] , group number:4=====
Predicting group 4
[(0, 17458), (1, 17458)]
===== Processing group:(28799.0, 35999.0] , group number:5=====
Predicting group 5
[(0, 30495), (1, 30495)]
===== Processing group:(35999.0, 43199.0] , group number:6=====
Predicting group 6
[(0, 47255), (1, 47255)]
===== Processing group:(43199.0, 50399.0] , group number:7=====
Predicting group 7
[(0, 62554), (1, 62554)]
===== Processing group:(50399.0, 57599.0] , group number:8=====
Predicting group 8
[(0, 78392), (1, 78392)]
===== Processing group:(57599.0, 64799.0] , group number:9=====
Predicting group 9
[(0, 94034), (1, 94034)]
===== Processing group:(64799.0, 71999.0] , group number:10=====
Predicting group 10
[(0, 1106

In [99]:
metric_data = list(zip(group_time_l,smote_acc_l,smote_pre_l,smote_rec_l,smote_f1_l,smote_tn_l,smote_fp_l,smote_fn_l,smote_tp_l))
df = pd.DataFrame(metric_data, columns =['period','Accuracy', 'Precision', 'Recall','F-1','TN','FP','FN','TP'])
df.to_csv(f"{result_dir}/dynamic_clf_smote_extra_feature_{results_name}.csv")
dfi.export(df.style.hide_index(), f"{result_dir}/dynamic_clf_smote_extra_feature_{results_name}.png")
df

/tmp/ipykernel_3720163/944619728.py:4: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  dfi.export(df.style.hide_index(), f"{result_dir}/dynamic_clf_smote_extra_feature_{results_name}.png")
[1113/141200.150361:WARNING:bluez_dbus_manager.cc(247)] Floss manager not present, cannot set Floss enable/disable.
[1113/141200.155137:WARNING:sandbox_linux.cc(380)] InitializeSandbox() called with multiple threads in process gpu-process.
[1113/141200.334911:INFO:headless_shell.cc(660)] Written to file /tmp/tmp7y2xw4mw/temp.png.


,period,Accuracy,Precision,Recall,F-1,TN,FP,FN,TP
0,"(21599.0, 28799.0]",0.996922,0.647059,0.846154,0.733333,5161,12,4,22
1,"(28799.0, 35999.0]",0.999234,0.916667,0.550000,0.687500,13036,1,9,11
2,"(35999.0, 43199.0]",0.999583,0.931818,0.911111,0.921348,16757,3,4,41
3,"(43199.0, 50399.0]",0.999217,0.687500,0.611111,0.647059,15294,5,7,11
4,"(50399.0, 57599.0]",0.999685,1.000000,0.814815,0.897959,15838,0,5,22
5,"(57599.0, 64799.0]",0.999553,1.000000,0.730769,0.844444,15642,0,7,19
6,"(64799.0, 71999.0]",0.999578,1.000000,0.681818,0.810811,16579,0,7,15
7,"(71999.0, 79199.0]",0.999364,0.750000,0.681818,0.714286,18848,5,7,15
8,"(79199.0, 86399.0]",0.999734,0.944444,0.850000,0.894737,15038,1,3,17
9,"(86399.0, 93599.0]",0.998082,0.538462,0.583333,0.560000,5717,6,5,7


In [100]:
smote_acc_l

[0.9969224850932872,
 0.9992341272880447,
 0.9995834573043737,
 0.999216556766991,
 0.9996848408446265,
 0.9995532295123819,
 0.9995783386542979,
 0.9993642384105961,
 0.9997343781127564,
 0.9980819529206626,
 0.9979550102249489,
 0.9987684729064039,
 0.9990235964198535,
 0.999846825457609,
 0.9995194906600997,
 0.9996767520041375,
 0.9995922171734825,
 0.9995870450120937,
 0.9995648660409026,
 0.9998716632443532,
 0.9992049469964664]